In [1]:
import pandas as pd

nodes_df = pd.read_csv('cifar_prunned_ranks.csv')
nodes_wide_df = nodes_df.pivot(index = 'filter_num',columns='class', values='prune_score')

classes = list(nodes_df['class'].unique())
classes.remove('overall')
classes.insert(0,'overall')

In [2]:
def get_layer(filter_num, df = nodes_df, col = 'layer'):
    return df.loc[(df['filter_num'] == filter_num) & (df['class'] == df['class'].unique()[0]), col].item()


nodes_wide_df.reset_index(inplace=True)

nodes_wide_df['layer'] = nodes_wide_df['filter_num'].apply(get_layer)



layers = {}
for index, row in nodes_df[nodes_df['class'] == 'overall'].iterrows(): 
    if row['layer'] not in layers:
        layers[row['layer']] = []
    layers[row['layer']].append(row['filter_num'])
nodes_df.tail(10)


num_layers = max(layers.keys()) + 1

#nodes_wide_df['filter_num_by_layer'] = nodes_wide_df.apply(lambda row: get_layer(row['filter_num'], col='filter_num_by_layer'), axis = 1)

In [3]:
import numpy as np
from sklearn import manifold
from sklearn.metrics import euclidean_distances

def add_norm_col(df,classes=classes):
    norms = []
    norm = 0
    for index, row in df.iterrows():
        for label in classes:
            norm += row[label]**2
        norm = np.sqrt(norm)
        norms.append(norm)
    norms = np.array(norms)
    df['class_norm'] = norms

add_norm_col(nodes_wide_df)   
    
layer_similarities = {}
for layer in layers:
    layer_df = nodes_wide_df[nodes_wide_df['layer'] == layer]
    for label in classes:
        layer_df[label] = layer_df.apply(lambda row : row[label]/row['class_norm'], axis = 1)   
    layer_similarities[layer] = euclidean_distances(layer_df.iloc[:,1:-2])



layer_mds = {}
for layer in layer_similarities:
	print('layer: %s'%str(layer))
	mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, 
      random_state=2, dissimilarity="precomputed", n_jobs=1)
	pos = mds.fit(layer_similarities[layer]).embedding_
	layer_mds[layer] = pos

#print(layer_mds)





/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

layer: 0
layer: 1
layer: 2
layer: 3


In [4]:
nodes_df[nodes_df['class'] == classes[0]].head(5)

,filter_num,layer,filter_num_by_layer,prune_score,class
1640,0,0,0,0.407439,overall
1641,1,0,1,0.047471,overall
1642,2,0,2,0.130462,overall
1643,3,0,3,0.106599,overall
1644,4,0,4,0.018162,overall


In [5]:
def layernum2name(layer,offset=1,title = 'layer'):
    return title+' '+str(layer+offset)

In [6]:
import numpy as np
import igraph as ig
import json
import urllib

N = len(nodes_wide_df.index)
sizes = list(nodes_wide_df['overall'])
format_sizes = []
for size in sizes:
    format_sizes.append(50*np.cbrt(size))
sizes = format_sizes

#N=len(data_dict['nodes'])


In [7]:
Edges_full = {}
for layer in layers:
    if layer+1 in layers:
        Edges_full[layer+1] = []
        for i in range(len(layers[layer])):
            for j in range(len(layers[layer+1])):
                Edges_full[layer+1].append((layers[layer][i],layers[layer+1][j],i,j))             
       
  
#print(Edges_full)

In [8]:
labels_list=(list(nodes_df[nodes_df['class'] == 'overall'].filter_num))
layers_list=(list(nodes_df[nodes_df['class'] == 'overall'].layer))
N = len(labels_list)


In [9]:
target_class = 'frog'

#Node Opacity
layer_colors = {0:'rgba(31,119,180,', 
                1:'rgba(255,127,14,',
                2:'rgba(44,160,44,', 
                3:'rgba(214,39,40,',
                4:'rgba(148, 103, 189,', 
                5:'rgba(140, 86, 75,',
                6:'rgba(227, 119, 194,',
                7:'rgba(127, 127, 127,',
                8:'rgba(188, 189, 34,',
                9:'rgba(23, 190, 207,'}


def color_scaling(x):
    return -(x-1)**4+1
    
node_colors_dict = {}
for layer in layers:
    node_colors_dict[layer] = []
    for node in layers[layer]:
        alpha = color_scaling(nodes_df[nodes_df['class']==target_class].iloc[node].prune_score)
        node_colors_dict[layer].append(layer_colors[layer]+str(round(alpha,3))+')')

#print(colors_dict)

In [10]:
#Node positions
layer_distance = 1   # distance in X direction each layer is separated by
node_positions = {}
layer_offset = 0
for layer in layer_mds:
    node_positions[layer] = {}
    node_positions[layer]['X'] = [] 
    node_positions[layer]['Y'] = [] 
    node_positions[layer]['Z'] = []  
    for i in range(len(layer_mds[layer])): 
        node_positions[layer]['Y'].append(layer_mds[layer][i][0])
        node_positions[layer]['Z'].append(layer_mds[layer][i][1])
        node_positions[layer]['X'].append(layer_offset)
    layer_offset+=1

#print(node_positions[0])

In [11]:
#Edge selection
edge_weights = {}
edge_threshold = .1
Edges = {}
for layer in Edges_full:
    Edges[layer] = []
    edge_weights[layer] = []
    for i in range(len(Edges_full[layer])):
        edge_weight = nodes_df[nodes_df['class']==target_class].iloc[Edges_full[layer][i][0]].prune_score*nodes_df[nodes_df['class']==target_class].iloc[Edges_full[layer][i][1]].prune_score
        if edge_weight > edge_threshold:
            Edges[layer].append(Edges_full[layer][i])
            edge_weights[layer].append(edge_weight)
            
#print(Edges)

#Edge Positions
edge_positions = {}
for layer in Edges:
    edge_positions[layer] = {}
    edge_positions[layer]['X'] = []
    edge_positions[layer]['Y'] = []
    edge_positions[layer]['Z'] = []
    for edge in Edges[layer]:
        edge_positions[layer]['X']+=([node_positions[layer-1]['X'][edge[2]],node_positions[layer]['X'][edge[3]], None])# x-coordinates of edge ends
        edge_positions[layer]['Y']+=([node_positions[layer-1]['Y'][edge[2]],node_positions[layer]['Y'][edge[3]], None])
        edge_positions[layer]['Z']+=([node_positions[layer-1]['Z'][edge[2]],node_positions[layer]['Z'][edge[3]], None])    

#print(edge_positions)


#Edge Colors
edge_colors_dict = {}
for layer in Edges:
    edge_colors_dict[layer] = []
    for weight in edge_weights[layer]:
        alpha = color_scaling(weight)
        edge_colors_dict[layer].append(layer_colors[layer]+str(round(alpha,3))+')')

In [77]:
#import chart_studio.plotly as py
import plotly.offline as py    #added
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)   #added

node_data = []
for layer in layers:
    #add nodes
    node_trace=go.Scatter3d(x=node_positions[layer]['X'],
               y=node_positions[layer]['Y'],
               z=node_positions[layer]['Z'],
               mode='markers',
               name=layernum2name(layer,title = 'nodes'),
               marker=dict(symbol='circle',
                             size=6,
                             opacity=.99,
                             color=node_colors_dict[layer],
                             #colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=.5)
                             ),
               text=layers[layer],
               hoverinfo='text'
               )
        
    node_data.append(node_trace)
    
edge_data = []    
for layer in edge_positions:        
    #add edges      
    edge_trace=go.Scatter3d(x=edge_positions[layer]['X'],
                            y=edge_positions[layer]['Y'],
                            z=edge_positions[layer]['Z'],
                            name=layernum2name(layer ,title = 'edges'),
                            mode='lines',
                            #line=dict(color=edge_colors_dict[layer], width=1.5),
                            line=dict(color='rgb(100,100,100)', width=1.5),
                            text = list(range(len(Edges[layer]))),
                            hoverinfo='text'
                            )
    edge_data.append(edge_trace)

 
combined_data = node_data+edge_data


#layout
axis=dict(showbackground=False,
          showspikes=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          #range=[0,0],
          title=''
          )

graph_layout = go.Layout(
         #title="%s through Prunned Cifar10 CNN"%target_class,
         #title = target_class,
         #width=1000,
         clickmode = 'event+select',
         transition = {'duration': 500},
         height=600,
         #showlegend=False,
         margin = dict(l=20, r=20, t=20, b=20),
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
         ),
         uirevision =  None   
         #hovermode='closest',
   )


fig=go.Figure(data=combined_data, layout=graph_layout)




In [70]:
## adding images
import glob
import os

node_image_directory = '/Users/chrishamblin/Desktop/graph_viz/node_images/'
list_of_node_images = [os.path.basename(x) for x in glob.glob('{}*.png'.format(node_image_directory))]

node_static_image_route = '/static_node/'


edge_image_directory = '/Users/chrishamblin/Desktop/graph_viz/edge_images/'
list_of_edge_images = [os.path.basename(x) for x in glob.glob('{}*.png'.format(edge_image_directory))]

edge_static_image_route = '/static_edge/'

In [79]:
import dash
import dash_core_components as dcc
import dash_html_components as html
#import utils.dash_reusable_components as drc
import flask
import os

from dash.dependencies import Input, Output, State


#external_stylesheets = ['https://codepen.io/amyoshino/pen/jzXypZ.css']
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(external_stylesheets = external_stylesheets)


styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}


app.layout = html.Div(
        [html.Div(         #Left side control panel
            children = [
             html.Label('Weighting Category'),
             dcc.Dropdown(
                options=[{'label': i, 'value': i} for i in classes],
                value='actgrads'
                ),           
             html.Label('Weighting Criterion'),
             dcc.Dropdown(
                options=[
                    {'label': 'Activations*Grads', 'value': 'actgrads'},
                    {'label': 'Activations', 'value': 'acts'}
                    
                ],
                value='actgrads'
                ),
                
             html.Label('Layer Projection'),
             dcc.Dropdown(
                options=[
                    {'label': 'MDS', 'value': 'MDS'},
                    {'label': 'Grid', 'value': 'grid'},
                    {'label': 'SOM', 'value': 'SOM'}
                ],
                value='MDS'
                ),

            html.Label('Multi-Select Dropdown'),
            dcc.Dropdown(
                options=[
                    {'label': 'New York City', 'value': 'NYC'},
                    {'label': u'Montréal', 'value': 'MTL'},
                    {'label': 'San Francisco', 'value': 'SF'}
                ],
                value=['MTL', 'SF'],
                multi=True
                ),

            html.Label('Radio Items'),
            dcc.RadioItems(
                options=[
                    {'label': 'New York City', 'value': 'NYC'},
                    {'label': u'Montréal', 'value': 'MTL'},
                    {'label': 'San Francisco', 'value': 'SF'}
                ],
                value='MTL'
                ),
                
            html.Label('Lower Edge Threshold'),
                dcc.Slider(
                    id='lower-thresh-slider',
                    min=0,
                    max=1,
                    step=0.01,
                    marks={i/10: str(i/10) for i in range(0,12,2)},
                    value=10,
                ),
                
            html.Label('Upper Edge Threshold'),
                dcc.Slider(
                    id='upper-thresh-slider',
                    min=0,
                    max=1,
                    step=0.01,
                    marks={i/10: str(i/10) for i in range(0,12,2)},
                    value=10,
                ),
                
            ], className="two columns"
        ),

        html.Div(
            children = [
                
            html.Div([
                dcc.Graph(
                    id='network-graph',
                    figure=fig
                )
            ], className= 'row'
            ),
                
            html.Div([
                html.Div([
                dcc.Dropdown(
                    id='node-image-dropdown',
                    options=[{'label': i, 'value': i} for i in list_of_node_images],
                    value=list_of_node_images[0]
                ),
                html.Img(id='node-image')
                ], className = "three columns"),
                
                html.Div([
                dcc.Dropdown(
                    id='edge-image-dropdown',
                    options=[{'label': i, 'value': i} for i in list_of_edge_images],
                    value=list_of_edge_images[0]
                ),
                html.Img(id='edge-image')
                ], className = "three columns")
                
             ], className= 'row'
             ),
                
                
            html.Div([
                html.Div([
                    dcc.Markdown("""
                        **Hover Data**

                        Mouse over values in the graph.
                    """),
                    html.Pre(id='hover-data', style=styles['pre'])
                ], className='two columns'),

                html.Div([
                    dcc.Markdown("""
                        **Click Data**

                        Click on points in the graph.
                    """),
                    html.Pre(id='click-data', style=styles['pre']),
                ], className='two columns'),

                html.Div([
                    dcc.Markdown("""
                        **Selection Data**

                        Choose the lasso or rectangle tool in the graph's menu
                        bar and then select points in the graph.

                        Note that if `layout.clickmode = 'event+select'`, selection data also 
                        accumulates (or un-accumulates) selected data if you hold down the shift
                        button while clicking.
                    """),
                    html.Pre(id='selected-data', style=styles['pre']),
                ], className='two columns'),

#                 html.Div([
#                     dcc.Markdown("""
#                         **Zoom and Relayout Data**

#                         Click and drag on the graph to zoom or click on the zoom
#                         buttons in the graph's menu bar.
#                         Clicking on legend items will also fire
#                         this event.
#                     """),
#                     html.Pre(id='relayout-data', style=styles['pre']),
#                 ], className='two columns')
                
                html.Div([
                    dcc.Markdown("""
                        **Figure Data**

                        Figure json info.
                    """),
                    html.Pre(id='figure-data', style=styles['pre']),
                ], className='four columns')
                
            ], className= 'row'
            )
        ], className="ten columns"
        )
    ]
)




####Call Back Functions

# @app.callback(
#     Output('figure-data', 'children'),
#     [Input('network-graph', 'figure')])
# def display_figure_data(figure):
#     return json.dumps(figure, indent=2)



@app.callback(
    Output('hover-data', 'children'),
    [Input('network-graph', 'hoverData')])
def display_hover_data(hoverData):
    return json.dumps(hoverData, indent=2)



@app.callback(
    Output('node-image-dropdown', 'value'),
    [Input('network-graph', 'clickData')])
def switch_node_image_click(clickData):
    if int(clickData['points'][0]['curveNumber']) >= num_layers:
        raise Exception('Do nothing, they clicked an edge')
    return list_of_node_images[int(clickData['points'][0]['text'])]


@app.callback(
    Output('edge-image-dropdown', 'value'),
    [Input('network-graph', 'clickData')])
def switch_edge_image_click(clickData):
    if int(clickData['points'][0]['curveNumber']) < num_layers:
        raise Exception('Do nothing, they clicked a node')
    return list_of_edge_images[int(clickData['points'][0]['pointNumber'])]



@app.callback(
    Output('network-graph', 'figure'),
    [Input('network-graph', 'clickData')])
def highlight_on_click(clickData):
    trace_num = int(clickData['points'][0]['curveNumber'])
    if trace_num < num_layers:   #highlight point
        for layer in node_colors_dict:
            if layer == trace_num:
                new_colors = list(node_colors_dict[trace_num])
                new_colors[clickData['points'][0]['pointNumber']] = 'rgba(0,0,0,1)'
                combined_data[trace_num]['marker']['color'] = new_colors
            else:
                combined_data[layer]['marker']['color'] = node_colors_dict[layer]
    else: #highlight edge
        #raise Exception('lets skip edges for now') 
        for layer in Edges:
            new_colors = list(['rgb(125,125,125)' for i in range(len(Edges[layer]))])
            #new_colors = edge_colors_dict[layer]
            if layer == trace_num-num_layers+1:
                new_colors[clickData['points'][0]['text']] = 'rgba(150,0,0,1)'
            combined_data[layer]['line']['color'] = new_colors
    
    layout = graph_layout
    layout['uirevision'] = True
    return {'data': combined_data,
            'layout': layout}



@app.callback(
    Output('click-data', 'children'),
    [Input('network-graph', 'clickData')])
def display_click_data(clickData):
    return json.dumps(clickData, indent=2)


@app.callback(
    Output('selected-data', 'children'),
    [Input('network-graph', 'selectedData')])
def display_selected_data(selectedData):
    return json.dumps(selectedData, indent=2)


# @app.callback(
#     Output('relayout-data', 'children'),
#     [Input('network-graph', 'relayoutData')])
# def display_relayout_data(relayoutData):
#     return json.dumps(relayoutData, indent=2)



#Node Images
@app.callback(
    Output('node-image', 'src'),
    [Input('node-image-dropdown', 'value')])
def update_node_image_src(value):
    return node_static_image_route + value

@app.server.route('{}<image_path>.png'.format(node_static_image_route))
def serve_node_image(image_path):
    image_name = '{}.png'.format(image_path)
    if image_name not in list_of_node_images:
        raise Exception('"{}" is excluded from the allowed static files'.format(image_path))
    return flask.send_from_directory(node_image_directory, image_name)


#Edge Images
@app.callback(
    Output('edge-image', 'src'),
    [Input('edge-image-dropdown', 'value')])
def update_edge_image_src(value):
    return edge_static_image_route + value

@app.server.route('{}<image_path>.png'.format(edge_static_image_route))
def serve_edge_image(image_path):
    image_name = '{}.png'.format(image_path)
    if image_name not in list_of_edge_images:
        raise Exception('"{}" is excluded from the allowed static files'.format(image_path))
    return flask.send_from_directory(edge_image_directory, image_name)



# Add a static image route that serves images from desktop
# Be *very* careful here - you don't want to serve arbitrary files
# from your computer or server

# @app.callback(Output('my-div', 'children'),
#                      [Input('my-input', 'value')],
#                      [State('my-div', 'children')])
# def update_div(value, existing_state):
#     if some_condition:
#          return existing_state


#py.iplot(fig, filename='small_net')



In [81]:
app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/May/2020 11:47:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:47:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:47:15] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:47:18] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/flask/app.py", line 1935, in d

127.0.0.1 - - [06/May/2020 11:47:18] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/flask/app.py", line 1935, in d

...

127.0.0.1 - - [06/May/2020 11:50:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [06/May/2020 11:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [06/May/2020 11:50:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:50:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [06/May/2020 11:52:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [06/May/2020 11:52:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 11:52:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.

In [55]:
fig.layout.scene

layout.Scene({
    'xaxis': {'showbackground': False,
              'showgrid': False,
              'showline': False,
              'showspikes': False,
              'showticklabels': False,
              'title': {'text': ''},
              'zeroline': False},
    'yaxis': {'showbackground': False,
              'showgrid': False,
              'showline': False,
              'showspikes': False,
              'showticklabels': False,
              'title': {'text': ''},
              'zeroline': False},
    'zaxis': {'showbackground': False,
              'showgrid': False,
              'showline': False,
              'showspikes': False,
              'showticklabels': False,
              'title': {'text': ''},
              'zeroline': False}
})

In [73]:
for layer in edge_colors_dict:
    print(layer)
    print(len(edge_colors_dict[layer]))
    print(edge_colors_dict[layer])

    
    
for layer in edge_positions:
    print(layer)
    print(edge_positions)
    
    

1
13
['rgba(150,0,0,1)', 'rgba(255,127,14,0.527)', 'rgba(255,127,14,0.646)', 'rgba(255,127,14,0.621)', 'rgba(255,127,14,0.629)', 'rgba(255,127,14,0.627)', 'rgba(255,127,14,0.385)', 'rgba(255,127,14,0.374)', 'rgba(255,127,14,0.771)', 'rgba(255,127,14,0.398)', 'rgba(255,127,14,0.407)', 'rgba(255,127,14,0.692)', 'rgba(255,127,14,0.348)']
2
20
['rgba(44,160,44,0.354)', 'rgba(44,160,44,0.395)', 'rgba(44,160,44,0.744)', 'rgba(44,160,44,0.427)', 'rgba(44,160,44,0.351)', 'rgba(44,160,44,0.417)', 'rgba(44,160,44,0.428)', 'rgba(44,160,44,0.425)', 'rgba(44,160,44,0.637)', 'rgba(44,160,44,0.63)', 'rgba(150,0,0,1)', 'rgba(44,160,44,0.408)', 'rgba(44,160,44,0.442)', 'rgba(44,160,44,0.444)', 'rgba(44,160,44,0.353)', 'rgba(44,160,44,0.379)', 'rgba(150,0,0,1)', 'rgba(44,160,44,0.388)', 'rgba(44,160,44,0.377)', 'rgba(150,0,0,1)']
3
13
['rgba(214,39,40,0.407)', 'rgba(214,39,40,0.38)', 'rgba(214,39,40,0.668)', 'rgba(214,39,40,0.368)', 'rgba(150,0,0,1)', 'rgba(214,39,40,0.369)', 'rgba(214,39,40,0.367)', 'r

In [75]:
layers

{0: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44],
 1: [45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67],
 2: [68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116],
 3: [117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  1

In [ ]:
#Hide zoom, pan thingy

dcc.Graph(
    id='my-graph',
    figure={'data': [{'x': [1, 2, 3]}]},
    config={
        'displayModeBar': False
    }
)

In [97]:
#image to numpy array
from PIL import Image
import numpy
im = Image.open("edge_images/0164.png").convert('LA')
np_im = numpy.array(im)
np_im = np_im[:,:,0]
np_im = np.flip(np_im,0)

#numpy array to plotly heatmap
fig = go.Figure(data=go.Heatmap(
                    z=np_im),
                layout=dict(
                    height=500,
                    width=500))


#fig['layout'].update(scene=dict(aspectmode="data"))
fig.show()
#im.show()